# SingularityDAO snapshot processing pipeline

This notebook it's meant to explore how the snapshots could be processed automatically.

[//]: <> (The code-only version can be found at `./scripts/pipeline.py`.)

## Notes

### Manually getting the data

Due to the fact that AGIX has less than 100,000 holders, we can manually download a .csv file with the balances of all holders [using Etherscan](https://etherscan.io/exportData?type=tokenholders&contract=0x5b7533812759b45c2b44c19e320ba2cd2681b542&decimal=8).

This is doable due to the small number of holders, but not really practical for automating the production pipeline. As this is just an example, it will work for our purposes here.

### Scalability

The main purpose of this notebook it's to explore how to automate the data processing of the snapshots.

That's the reason why we are only dealing with a subset of the snapshots.

Once the process works reliably and can be performed automatically, the next step would be to containerize it for cloud processing.

In principle, the processing could be distributed by grouping all the addresses to process in batches of N addresses, and each container would calculate the eligibility for one batch at a time. Once the batch it's processed, the container would either request another one, or be destroyed if there aren't more.

This shouldn't be necessary if the amount of the data it's relatively small and the calculations can be done in a single machine quickly.

## 1. Take snapshots

For the example pipeline, I'm going to use just a small number of snapshots taken manually, due to the fact that the main focus of this notebook is to create the processing pipeline, not to gather the snapshots.

The first step is to import all the libraries that we're going to use for the data processing and for gathering insights about the dataset with statistical analysis.

In [1]:
# Import libraries
# Pandas for tabular data
import pandas as pd
import numpy as np
from os import walk
from pprint import pprint

Next step is reading the `data` directory to see how many snapshots we have for each token respectively (AGI and AGIX)

In [2]:
def get_snapshots(path):
    return next(walk(path), (None, None, []))[2]

agi_snapshots_files = get_snapshots('../data/holders/agi')
agix_snapshots_files = get_snapshots('../data/holders/agix')

pprint(agi_snapshots_files)
pprint(agix_snapshots_files)

['12700788-export-tokenholders-for-contract-0x8eb24319393716668d768dcec29356ae9cffe285.csv']
['12700800-export-tokenholders-for-contract-0x5b7533812759b45c2b44c19e320ba2cd2681b542.csv',
 '12709185-export-tokenholders-for-contract-0x5b7533812759b45c2b44c19e320ba2cd2681b542.csv',
 '12709949-export-tokenholders-for-contract-0x5b7533812759b45c2b44c19e320ba2cd2681b542.csv']


Load snapshots for liquidity providers and stakers.

In this example, the snapshots are the same as the AGIX holders, to focus on developing the calculations first, leaving getting the data from the database for later on.

In [13]:
lp_snapshots_files = get_snapshots('../data/lp')

stakers_snapshots_files = get_snapshots('../data/stakers')

Read the snapshots' contents using `pandas` and convert balances to unsinged long numbers (SDAO wei)

In [14]:
# Decimals AGI/AGIX

DECIMALS_AGI = 8

CONVERT_TO_FULL_BALANCE_AGI = np.power(np.ulonglong(10), DECIMALS_AGI)

AGI_THRESHOLD = 1000 * CONVERT_TO_FULL_BALANCE_AGI

def read_csv(folder, file):
    # Read csv file
    data_frame = pd.read_csv('../data/%s/%s' % (folder, file))
    # Sort accounts by holding amount, larger holders at the top
    data_frame = data_frame.astype({'Balance': np.ulonglong})
    data_frame['Balance'] = data_frame['Balance'].apply(lambda x: x * CONVERT_TO_FULL_BALANCE_AGI)
    data_frame = data_frame.sort_values('Balance', ascending=False)
    return data_frame

agi_snapshots_raw = [read_csv("holders", "agi/" + file) for file in agi_snapshots_files]
agix_snapshots_raw = [read_csv("holders", "agix/" + file) for file in agix_snapshots_files]

lp_snapshots_raw = [read_csv("lp", file) for file in lp_snapshots_files]
stakers_snapshots_raw = [read_csv("stakers", file) for file in stakers_snapshots_files]

The snapshots are now loaded as a panda DataFrame.

Let's see the structure of a single snapshot and a single row, to get a better idea of the dataset.

In [15]:
print(agi_snapshots_raw[0].columns)
print(agi_snapshots_raw[0].iloc[0])

Index(['HolderAddress', 'Balance', 'PendingBalanceUpdate'], dtype='object')
HolderAddress           0xbe0eb53f46cd790cd13851d5eff43d12404d33e8
Balance                                        14182336000000000.0
PendingBalanceUpdate                                            No
Name: 8470, dtype: object


Let's remove the `PendingBalanceUpdate` column and rename the other two, to clean the dataset and make it more practical.

In [16]:
def clear_snapshot(snapshot):
    cleaned_snapshot = snapshot.drop('PendingBalanceUpdate', axis="columns")
    cleaned_snapshot = cleaned_snapshot.rename(columns={"HolderAddress": "address", "Balance": "balance"})
    cleaned_snapshot = cleaned_snapshot.reset_index(drop=True)
    return cleaned_snapshot

agi_snapshots = [clear_snapshot(snapshot) for snapshot in agi_snapshots_raw]
agix_snapshots = [clear_snapshot(snapshot) for snapshot in agix_snapshots_raw]

lp_snapshots = [clear_snapshot(snapshot) for snapshot in lp_snapshots_raw]
stakers_snapshots = [clear_snapshot(snapshot) for snapshot in stakers_snapshots_raw]

print(agi_snapshots[0].columns)
# Address and balance from the account with the largest holding, one of the Binance wallets
print(agi_snapshots[0].iloc[0])

Index(['address', 'balance'], dtype='object')
address    0xbe0eb53f46cd790cd13851d5eff43d12404d33e8
balance                           14182336000000000.0
Name: 0, dtype: object


## 2. Calculate eligibility

With the snapshot data ready, we can start to calculate the eligible addresses.

Note that the following is intended to illustrate the process, but with the actual dataset of snapshots, a similar but more complex processing would be applied.

Additionally, for the sake of the example, this process will only take into account holders, ignoring stakers and liquidity providers.

Nonetheless, it's feasible to adapt this same series of steps to stakers and liquidity providers, by preparing their respective datasets as described in step 1.

### Initial snapshot

There's an initial snapshot that delimits how many addresses are eligible for the airdrop.

In my case it's the snapshot of the frozen AGI balances, but in the airdrop it would be the snapshot from 17th of April 2021, at 23:59 UTC+0.

Let's create a subset based on the addresses from the first snapshot that have more than 1.000 AGI.

In [22]:
print("AGI Snapshots: %s" % len(agi_snapshots))
print("AGIX Snapshots: %s" % len(agix_snapshots))
print("LP Snapshots: %s" % len(lp_snapshots))
print("Stakers Snapshots: %s" % len(stakers_snapshots))
print()

# Get the first snapshot and use it as the starting point for the calculations
def get_initial(initial_snapshot, category):
    total_addresses = len(initial_snapshot.index)
    eligible_addresses_initial = initial_snapshot[initial_snapshot['balance'] >= AGI_THRESHOLD]
    
    print('Total Addresses (%s): %s' % (category, total_addresses))
    print('Eligible Addresses (%s): %s' % (category, len(eligible_addresses_initial.index)))
    print()
    
    return eligible_addresses_initial

eligible_addresses_holders = get_initial(agi_snapshots[0], 'holders')
eligible_addresses_lp = get_initial(lp_snapshots[0], 'LP')
eligible_addresses_stakers = get_initial(stakers_snapshots[0], 'stakers')

print()
# Print address with smaller eligible balance
print(eligible_addresses_holders.iloc[-1])

AGI Snapshots: 1
AGIX Snapshots: 3
LP Snapshots: 2
Stakers Snapshots: 2

Total Addresses (holders): 25979
Eligible Addresses (holders): 16689

Total Addresses (LP): 24414
Eligible Addresses (LP): 16470

Total Addresses (stakers): 24414
Eligible Addresses (stakers): 16470


address    0xba3699d5fc6276e87c3a761eaaf82162d4a86854
balance                                100000000000.0
Name: 16688, dtype: object


We can see that from the initial ~26k addresses, only 16689 pass the threshold to be eligible.

Now, it's a matter of iterating through the remaining snapshots using this initial set of accounts, and checking if the accounts are still eligible, removing the ones that are below the threshold.

### Iterate through the snapshots

First, let's merge all snapshots (AGI and AGIX) into a single array and discard the first one, as that one it's already processed.

In [8]:
# Merge snapshots
holders_snapshots = agi_snapshots + agix_snapshots

Total snapshots (including first one): 4
Total snapshots (without including first one): 3


Now, we iterate over the snapshots, filtering the initial set of eligible accounts.

In [23]:
def filter_addresses(initial_df, snapshot_df):
    # Calculate intersection of eligible addresses between existing set and snapshot set
    initial_set = set(initial_df['address'])
    snapshot_set = set(snapshot_df['address'])
    addresses_intersection = list(initial_set.intersection(snapshot_set))
    
    # Filter addresses based on whether they're contained on the intersection set or not
    filtered_df = initial_df[initial_df.apply(lambda x: x['address'] in addresses_intersection, axis=1)]
    
    return filtered_df

def get_eligible(initial_df, snapshots, category):
    print()
    print('Initial Eligible Addresses (%s): %s' % (category, len(initial_df.index)))
    print()

    eligible_df = initial_df

    for index, snapshot in enumerate(snapshots):
        print('Snapshot #%s' % index)
        snapshot_eligible = snapshot[snapshot['balance'] >= AGI_THRESHOLD]
        print('Eligible Addresses from snapshot: %s addresses' % len(snapshot_eligible.index))
        eligible_df = filter_addresses(eligible_df, snapshot_eligible)
        print('Eligible Addresses: %s' % len(eligible_df.index))
        print()

    print('Total Eligible Addresses (%s): %s' % (category, len(eligible_df.index)))
    

eligible_addresses_holders = get_eligible(eligible_addresses_holders, holders_snapshots, 'holders')
eligible_addresses_lp = get_eligible(eligible_addresses_lp, lp_snapshots, 'LP')
eligible_addresses_stakers = get_eligible(eligible_addresses_stakers, stakers_snapshots, 'stakers')

Initial Eligible Addresses (holders): 16689

Snapshot #0
Eligible Addresses from snapshot: 16470 addresses
Eligible Addresses: 15737

Snapshot #1
Eligible Addresses from snapshot: 16470 addresses
Eligible Addresses: 15717

Snapshot #2
Eligible Addresses from snapshot: 16473 addresses
Eligible Addresses: 15715

Total Eligible Addresses (holders): 15715
Initial Eligible Addresses (LP): 16470

Snapshot #0
Eligible Addresses from snapshot: 16470 addresses
Eligible Addresses: 16470

Snapshot #1
Eligible Addresses from snapshot: 16473 addresses
Eligible Addresses: 16467

Total Eligible Addresses (LP): 16467
Initial Eligible Addresses (stakers): 16470

Snapshot #0
Eligible Addresses from snapshot: 16470 addresses
Eligible Addresses: 16470

Snapshot #1
Eligible Addresses from snapshot: 16473 addresses
Eligible Addresses: 16467

Total Eligible Addresses (stakers): 16467


### Calculating airdrop amount

#### Parameters

In [ ]:
# Decimals SDAO

DECIMALS_SDAO = 18

CONVERT_TO_FULL_BALANCE_SDAO = np.power(np.ulonglong(10), DECIMALS_SDAO)

# Reward parameters

TOTAL_STAKING_REWARD = 550000.0

TOTAL_REWARD = 825000.0

# Adjust rewards to be full balance

TOTAL_STAKING_REWARD *= CONVERT_TO_FULL_BALANCE_SDAO

TOTAL_REWARD *= CONVERT_TO_FULL_BALANCE_SDAO

#### Stakers

There are two kinds of rewards for stakers:
- Per user (divided equally among staking wallets)
- Per stake amount (delivered proportionally to the amounts staked)

#### Holders and LP

Knowing the eligibility of the addresses, we can calculate the balances now using the following formula.

With those premises in place, we can calculate the final reward for each user by using the following formula

`Reward = total_reward * log10(1+user_balance) / SUM(log10(1+user_balance))`

In [11]:
# Define SUM(log10(1+user_balance)) as a constant variable

balances = list(eligible_addresses['balance'])

balances_log10 = [np.log10(1 + (balance)) for balance in balances]

sum_balances_log10 = np.sum(balances_log10)

# Define the function that calculates the reward for each user

def calculate_holder_reward(total_reward, user_balance_index):
    user_balance_log10 = balances_log10[user_balance_index]
    reward_percentage = np.double(user_balance_log10) / np.double(sum_balances_log10)
    # Calculate reward and convert to final balance
    # Move the position of the floating point to have more precision around small percentages
    return (total_reward * (reward_percentage * np.power(10, 9)) / np.power(10, 9)) / CONVERT_TO_FULL_BALANCE_SDAO


# Calculate rewards and add the SDAO value as a column to the DateFrame

holder_rewards = [calculate_holder_reward(TOTAL_REWARD, index) for index, balance in enumerate(balances)]

rewards_df = eligible_addresses.copy()

rewards_df.insert(0, 'holder_reward', holder_rewards)

rewards_df['balance'] = rewards_df['balance'].apply(lambda x: x / CONVERT_TO_FULL_BALANCE_AGI)

print(rewards_df)

# Verify that the total amount of allocated reward matches the expected value

calculated_holder_reward = np.sum(list(rewards_df['holder_reward']))

adjusted_total_holder_reward = (TOTAL_REWARD / CONVERT_TO_FULL_BALANCE_SDAO)

print('Allocated reward (holders and LP): %s' % adjusted_total_holder_reward)
print('Calculated reward (holders and LP): %s' % calculated_holder_reward)

if np.ulonglong(calculated_holder_reward) != adjusted_total_holder_reward:
    raise Exception('Error calculating rewards', 'final reward sum does not match allocated reward')

       holder_reward                                     address      balance
0          72.108259  0xbe0eb53f46cd790cd13851d5eff43d12404d33e8  141823360.0
1          71.509113  0xf977814e90da44bfa03b6295a0616a897441acec  104122164.0
4          68.034971  0xa1d8d972560c2f8144af871db508f0b0b10a3fbf   17352399.0
5          67.739481  0x19184ab45c40c2920b0e0e31413b9434abd243ed   14899506.0
6          67.695774  0x8699b0ffff9136df5fed0175baf4b65477378a3d   14567393.0
...              ...                                         ...          ...
16684      49.108670  0xc4b4e449432171c7b945c4a8af36b107f8d70df4       1000.0
16685      49.108670  0xa6e615df1824cc71eb7ae5a823c5808ea2b196fc       1000.0
16686      49.108670  0x369fb3e7cebc8e4bc2d6166f9e5139d564c38db4       1000.0
16687      49.108670  0x1da14e9cdae510ae75972f5221200fa781fdb4c8       1000.0
16688      49.108670  0xba3699d5fc6276e87c3a761eaaf82162d4a86854       1000.0

[15715 rows x 3 columns]
Allocated reward (holders and LP): 825

### Adding unclaimed balances

The missing step would be sum all the unclaimed amount from previous airdrops, for this example that's not possible with the data at hand though.